# ***Distribución BAÑOS***

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and "puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
    and ("jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [7]:
df_may_2024_puebla.columns

Index(['id', 'Q', 'categoria', 'ids', 'Propiedad', 'Precio', 'Metros_total',
       'm2_construido', 'Precio_M2_Terreno', 'Precio_M2_Construido',
       'Personas_interesadas', 'Tiempo_publicacion',
       'Promedio_Interes_Mensual', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Cantidad_de_pisos',
       'Antiguedad', 'Seguridad', 'Fraccionamiento', 'Colonia', 'CP', 'url'],
      dtype='object')

In [8]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [9]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Total de baños

In [10]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)
        df['baño_total'] = df['baño_total'].clip(upper=8) # Limitar los valores de 'baño_total' a un máximo de 8
        #df['baño_total'] = pd.cut(df['baño_total'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

Nombre del DataFrame: df_feb_2024_puebla
baño_total    1    2    3    4   5   6  7  8
segmento                                    
A           178  215   93   44  14   0  0  4
B           183  292  119   18   2   0  1  0
C           244  264   74    8   1   1  0  0
D           331  261   28    4   0   0  0  1
E           566  250  139   18   3   4  4  0
L            38   36   40   30  18   5  1  3
S           321  206  200  113  49  16  6  8
Nombre del DataFrame: df_jul_2023_puebla
baño_total   0   1    2    3    4   5   6  7  8
segmento                                       
A            3   8   85  122   85  20   4  1  1
B            4  19  116  171   63  11   2  0  0
C            6  14  165  128   23   1   2  0  0
D            2  58  200   56    5   0   0  0  0
E            2  63   37    8    9   6   3  0  0
L            3   0   11   10   30  11  12  9  7
S           13  10   88  143  168  94  30  7  4
Nombre del DataFrame: df_mar_2024_puebla
baño_total  1.0  2.0  2.5  3.0  4.0  5.0

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18840\705114878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18840\705114878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18840\705114878.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [11]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total   0.0     1.0  1.5     2.0  2.5    3.0    4.0  4.5    5.0    6.0  \
segmento                                                                      
A            5.0   607.0  0.0   812.0  0.0  554.0  277.0  0.0   71.0   11.0   
B            9.0   629.0  0.0  1014.0  0.0  636.0  204.0  0.0   48.0   10.0   
C           10.0   791.0  0.0  1114.0  1.0  428.0   78.0  0.0   15.0    3.0   
D            7.0  1121.0  0.0  1051.0  0.0  176.0   23.0  0.0    4.0    1.0   
E            2.0  1827.0  1.0   695.0  0.0  368.0   76.0  0.0   20.0   10.0   
L            3.0   171.0  NaN   100.0  0.0  114.0  128.0  NaN   64.0   37.0   
S           17.0  1087.0  0.0   711.0  0.0  828.0  682.0  1.0  349.0  113.0   

baño_total   7.0   8.0  
segmento                
A            8.0   8.0  
B            5.0   0.0  
C            0.0   2.0  
D            0.0   3.0  
E            6.0   4.0  
L           20.0  13.0  
S           31.0  28.0  


In [12]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    guardar_grafico_como_html(fig, 'g_bar_baños', carpeta='assets/graficas') # Para guardar la gráfica como archivo HTML
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Número de baños",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )    
    
    fig.show()    
    return fig

# Guardar la gráfica como archivo HTML en la carpeta especificada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total   0.0     1.0  1.5     2.0  2.5    3.0    4.0  4.5    5.0    6.0  \
segmento                                                                      
A            5.0   607.0  0.0   812.0  0.0  554.0  277.0  0.0   71.0   11.0   
B            9.0   629.0  0.0  1014.0  0.0  636.0  204.0  0.0   48.0   10.0   
C           10.0   791.0  0.0  1114.0  1.0  428.0   78.0  0.0   15.0    3.0   
D            7.0  1121.0  0.0  1051.0  0.0  176.0   23.0  0.0    4.0    1.0   
E            2.0  1827.0  1.0   695.0  0.0  368.0   76.0  0.0   20.0   10.0   
L            3.0   171.0  NaN   100.0  0.0  114.0  128.0  NaN   64.0   37.0   
S           17.0  1087.0  0.0   711.0  0.0  828.0  682.0  1.0  349.0  113.0   

baño_total   7.0   8.0  
segmento                
A            8.0   8.0  
B            5.0   0.0  
C            0.0   2.0  
D            0.0   3.0  
E            6.0   4.0  
L           20.0  13.0  
S           31.0  28.0  


In [13]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
print("Tabla general total de registros:")
print(tabla_suma)


Tabla general total de registros:
baño_total
0.0      53.0
1.0    6233.0
1.5       1.0
2.0    5497.0
2.5       1.0
3.0    3104.0
4.0    1468.0
4.5       1.0
5.0     571.0
6.0     185.0
7.0      70.0
8.0      58.0
dtype: float64


In [14]:
# Calcular el total de registros
total_registros = tabla_suma.sum()
# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (tabla_suma / total_registros) * 100

# Imprimir los porcentajes
print("Porcentaje de cada número de baños único en relación con el total de registros:", total_registros)
print(porcentajes)


Porcentaje de cada número de baños único en relación con el total de registros: 17242.0
baño_total
0.0     0.307389
1.0    36.150099
1.5     0.005800
2.0    31.881452
2.5     0.005800
3.0    18.002552
4.0     8.514093
4.5     0.005800
5.0     3.311681
6.0     1.072961
7.0     0.405985
8.0     0.336388
dtype: float64


In [15]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Lista de colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

# Función para sumar tablas
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Función para calcular porcentajes
def calcular_porcentajes(tabla):
    # Calcular los porcentajes
    tabla_porcentajes = tabla / tabla.sum() * 100
    return tabla_porcentajes

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
porcentajes = calcular_porcentajes(tabla_suma) # Calcular porcentajes

# Crear el gráfico de pastel con colores especificados
figURE = go.Figure(data=[go.Pie(labels=porcentajes.index, values=porcentajes, sort=False, marker=dict(colors=colores))])
figURE.update_layout( title_x=0.5, title_font_size=20,margin=dict(l=10, r=10, t=10, b=10),  )
figURE.update_traces(rotation=45) # Ángulo de inicio de las porciones del gráfico

# Exportar gráfica como archivo HTML y mostrar la figura
def guardar_grafico_como_html(figURE, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(figURE, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(figURE, 'g_pie_baños', carpeta='assets/graficas')

# Mostrar la figura
figURE.show()